In [ ]:
# Import libraries
from bs4 import BeautifulSoup
import requests
import time
import datetime
import smtplib
import csv
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error





In [ ]:
# Connect to Website and pull in data

URL = 'https://www.amazon.com/Funny-Data-Systems-Business-Analyst/dp/B07FNW9FGJ/ref=sr_1_3?dchild=1&keywords=data%2Banalyst%2Btshirt&qid=1626655184&sr=8-3&customId=B0752XJYNL&th=1'

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

page = requests.get(URL, headers=headers)

soup1 = BeautifulSoup(page.content, "html.parser")

soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

title = soup2.find(id='productTitle').get_text()

price = soup2.find(id='priceblock_ourprice').get_text()


print(title)
print(price)

AttributeError: 'NoneType' object has no attribute 'get_text'

In [ ]:
price = price.strip()[1:]  # Remove leading/trailing white spaces and the dollar sign
title = title.strip()  # Remove leading/trailing white spaces

# Print the cleaned title and price
print(title)
print(price)

# Get the current date
today = datetime.date.today()

# Print the current date
print(today)


In [ ]:
# Create a CSV file and write the headers and data
header = ['Title', 'Price', 'Date']  # Define the headers for the CSV file
data = [title, price, today]  # Create a list of data to be written

# Open the CSV file in write mode and write the headers and data
with open('AmazonWebScraperDataset.csv', 'w', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerow(data)

# Read the CSV file using pandas and print its contents
df = pd.read_csv(r'C:\Users\alexf\AmazonWebScraperDataset.csv')
print(df)


In [ ]:
with open('AmazonWebScraperDataset.csv', 'a+', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(data)

In [ ]:
#Combine all of the above code into one function


def check_price():
    URL = 'https://www.amazon.com/Funny-Data-Systems-Business-Analyst/dp/B07FNW9FGJ/ref=sr_1_3?dchild=1&keywords=data%2Banalyst%2Btshirt&qid=1626655184&sr=8-3&customId=B0752XJYNL&th=1'

    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    page = requests.get(URL, headers=headers)

    soup1 = BeautifulSoup(page.content, "html.parser")

    soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

    title = soup2.find(id='productTitle').get_text()

    price = soup2.find(id='priceblock_ourprice').get_text()

    price = price.strip()[1:]
    title = title.strip()

    import datetime

    today = datetime.date.today()

    import csv

    header = ['Title', 'Price', 'Date']
    data = [title, price, today]

    with open('AmazonWebScraperDataset.csv', 'a+', newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(data)

In [ ]:
while True:
    check_price()
    time.sleep(86400)  # Wait for 24 hours (86400 seconds) before the next execution

# Read the updated CSV file using pandas and print its contents
df = pd.read_csv(r'C:\Users\alexf\AmazonWebScraperDataset.csv')
print(df)

In [ ]:
# Feature engineering and model training for time-series forecasting
# Assuming the scraped data is stored in 'df' DataFrame
X = df[['Price']]  # Use price as the feature
y = df['Price'].shift(-7)  # Target variable is the price 7 days ahead

# Split the data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Create and train the XGBoost model
model = XGBRegressor()
model.fit(X_train, y_train)

# Make recursive predictions for the next 7 days
predictions = []
last_price = X_test.iloc[-1]  # Start with the last known price
for _ in range(7):
    prediction = model.predict(last_price.values.reshape(1, -1))
    predictions.append(prediction[0])
    last_price = prediction  # Update the last price with the predicted value

# Print the predicted prices for the next 7 days
print("Predicted prices for the next 7 days:")
for i, price in enumerate(predictions):
    print(f"Day {i+1}: {price}")